In [7]:
import pandas as pd
import os

Import all the files

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
ds_path = '/content/drive/MyDrive/Codecademy/Page_Visits_Funnel_Project'
visits = pd.read_csv(os.path.join(ds_path , 'visits.csv'),
                     parse_dates=[1])
cart = pd.read_csv(os.path.join(ds_path, 'cart.csv'),
                   parse_dates=[1])

checkout = pd.read_csv(os.path.join(ds_path,'checkout.csv'),
                       parse_dates=[1])
purchase = pd.read_csv(os.path.join(ds_path,'purchase.csv'),
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [12]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart

In [121]:
visits_carts = visits.merge(cart, how='left')
print(visits_carts.head())
print(len(visits_carts))

                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time  
0                 NaT  
1 2017-01-26 14:44:00  
2 2017-08-20 08:31:00  
3                 NaT  
4                 NaT  
2000


Step 3: How long is `visits_cart`?

In [114]:
#visits_carts.user_id.count()
total_visits = len(visits_carts)
print(len(visits_carts))

2000


Step 4: How many timestamps are null for `cart_time`?

In [117]:
#null_carts = visits_carts[visits_carts.cart_time.isnull()].count()
null_carts = len(visits_carts[visits_carts.cart_time.isnull()])
print(null_carts)

1652


Step 5: What percentage only visited?

In [118]:
#(visits_carts[visits_carts.cart_time.isnull()].count()) / len(visits_carts)
visit_no_cart = float(null_carts) / float(total_visits)
print(visit_no_cart)

0.826


Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [119]:
cart_checkout = cart.merge(checkout, how='left')
null_checkout_time = len(cart_checkout[cart_checkout.checkout_time.isnull()])
cart_no_checkout = float(null_checkout_time) / float(len(cart))
print(cart_no_checkout)

0.3505747126436782


Step 7: Merge it all together

In [120]:
all_df = visits_carts.merge(cart_checkout, how='left').merge(purchase, how='left')
print(all_df.head())
print(len(all_df))

                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time       checkout_time       purchase_time  
0                 NaT                 NaT                 NaT  
1 2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00  
2 2017-08-20 08:31:00                 NaT                 NaT  
3                 NaT                 NaT                 NaT  
4                 NaT                 NaT                 NaT  
2108


Step 8: % of users who got to checkout but did not purchase

In [135]:
reached_checkout = all_df[~all_df.checkout_time.isnull()]
checkout_no_purchase = all_df[(all_df.purchase_time.isnull()) & (~all_df.checkout_time.isnull())]
percent_no_purchase = float(len(checkout_no_purchase)) / float(len(reached_checkout))
print('% of users who got to checkout but did not purchase:', percent_no_purchase)

% of users who got to checkout but did not purchase: 0.24550898203592814


Step 9: check each part of the funnel, let's print all 3 of them again

In [145]:
funnel_1 = (" {}% of users who visited but didn't add anything to their cart \n".format(round(visit_no_cart*100,2)))
funnel_2 = ("{}% of users who added to their cart but didn't checkout \n".format(round(cart_no_checkout*100,2)))
funnel_3 = ("{}% of users checked out but didn't make a purchase \n".format(round(percent_no_purchase*100,2)))
print(funnel_1,funnel_2,funnel_3)

 82.6% of users who visited but didn't add anything to their cart 
 35.06% of users who added to their cart but didn't checkout 
 24.55% of users checked out but didn't make a purchase 



*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [56]:
all_df['purchase_vs_visit_time'] = all_df.purchase_time - all_df.visit_time

Step 11: examine the results

In [57]:
print(all_df.purchase_vs_visit_time)

0     0 days 00:44:00
1     0 days 00:38:00
2     0 days 00:41:00
3     0 days 01:00:00
4     0 days 01:00:00
            ...      
247   0 days 00:29:00
248   0 days 00:46:00
249   0 days 00:32:00
250   0 days 00:28:00
251   0 days 00:11:00
Name: purchase_vs_visit_time, Length: 252, dtype: timedelta64[ns]


Step 12: average time to purchase

In [59]:
print(all_df.purchase_vs_visit_time.mean())

0 days 00:43:12.380952380
